In [1]:
from selenium import webdriver
options = webdriver.ChromeOptions() 
options.add_argument('--headless') 
options.add_argument('--no-sandbox') 
options.add_argument('--disable-dev-shm-usage') 
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth', None)
import urllib.request
import time
import datetime

In [2]:
def start_crawling(keyword,START_DATE,END_DATE):
    origin_URL = (f'ttp://search.khan.co.kr/search.html?stb=khan&dm=5&d1={start_date}~{end_date}&q=') # start_date와 end_date넣어주어서 범위지정해줍니다.
    URL = origin_URL + keyword + '&pg=' # 링크 만들어서 getlink로 보내줍니다.
    get_link(URL)
    
def get_link(hURL):
    firstURL = "h" + hURL # 하이퍼링크이슈가 있어서 'h' 제거후 붙인다음 마지막에 붙여주었습니다
    driver = webdriver.Chrome(executable_path=r'C:/chromedriver/chromedriver.exe')
    driver.implicitly_wait(3)
    driver.get(firstURL)

    firstsoup = BeautifulSoup(driver.page_source, 'html.parser') # 첫화면에서 soup따주고
    num1 = firstsoup.select('#container > div.content > div.news.section > h3 > span')
    num2 = num1[0].get_text()#날자 따준다음
    num3 = num2.replace('(총 ', '')
    num4 = num3.replace(' 건 검색)', '')
    article_num = int(num4.replace(',', '')) # (총 xx,xxx 건 검색) 전처리 이후 int로 변환해줍니다.

    page_num = int((article_num / 10) +1) # page수 구해준후
    
    for i in range(1,page_num+1):# for문에서 page수 만큼 반복시켜줍니다. 
        URL = firstURL + str(i)
        driver.get(URL)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        for i in range(2,12): # 마지막 페이지에선 갯수가 모잘라서 에러가 납니다. 방지하기 위해 try, except문으로 마지막페이지 에러방지했습니다.
            try:
                link = soup.select(f'#container > div.content > div.news.section > dl:nth-child({i}) > dt > a')[0]['href'][:-12]
                # 링크 따주고
                LINK_OF_ARTICLE.append(link) # 어펜드 먼저 해줍니다.
                
            except IndexError as ERR:
                pass


            
def get_text(link): #링크 받아서
    req = urllib.request.Request(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'})        
    source_code_from_url = urllib.request.urlopen(req)
    articlesoup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8') # soup 따주고
    
    try:
        title = articlesoup.select('#article_title')[0].get_text()
        # title을 따주는데 가끔 에러가 발생합니다. ("div.highlight"가 있는경우) 워낙 많은양의 기사를 가져오다보니 과감히 배제하였습니다.
    except:
        title = "ERROR"
   
    try:
        date = articlesoup.select('#container > div.art_header.borderless > div.function_wrap > div.pagecontrol > div > em:nth-child(1)')[0].get_text()[5:15]
        # date도 마찬가지로 ("div.highlight"가 있는경우) 에러가 발생하는데, 극히 일부분이기에 과감히 배제하였습니다.
        # 또한 ERROR 를 추가해 전처리 과정에서 행을 삭제하였습니다.
    except:        
        date = "ERROR"
        
    all_article = '' # 기사 본문 생성하고
    for article in articlesoup.select('p.content_text'):
        all_article += article.get_text() # 하나 하나 붙여줍니다.
        all_article += '.' # 중간중간 '.'을 추가해 단어의 합체를 막았습니다.
        # ex) 반도체 역할 톡톡히
        #
        #     삼성에서는 이러한~~              ->반도체 역할 톡톡히삼성에서는  ->  반도체 역할 톡톡히.삼성에서는
        
        
    TITLE_OF_ARTICLE.append(title)
    DATE_OF_ARTICLE.append(date)
    CONTENT_OF_ARTICLES.append(all_article) #어펜드 해줍니다.

In [3]:
start_date = '20100101'
end_date = '20201231'

keyword = '인공지능'
TITLE_OF_ARTICLE = []
DATE_OF_ARTICLE = []
CONTENT_OF_ARTICLES = []
LINK_OF_ARTICLE = []

#전역변수 초기화 및 날짜, 키워드 설정

starttime = time.time()
start_crawling(keyword,start_date,end_date) #링크만 먼저 따줍니다.

#이후 링크에서 링크를 하나씩 가져와서 get_text해줍니다.
for i in range(0,len(LINK_OF_ARTICLE)):
    link = LINK_OF_ARTICLE[i]
    get_text(link)
    
    if i % 100 == 0: # 백번째 마다 완료문구 나오게 설정했습니다.
        print(f'{i}번째 기사 완료')

runtime = time.time() - starttime
print(f'끝! 걸린시간 = {runtime}초')


0번째 기사 완료
100번째 기사 완료
200번째 기사 완료
300번째 기사 완료
400번째 기사 완료
500번째 기사 완료
600번째 기사 완료
700번째 기사 완료
800번째 기사 완료
900번째 기사 완료
1000번째 기사 완료
1100번째 기사 완료
1200번째 기사 완료
1300번째 기사 완료
1400번째 기사 완료
1500번째 기사 완료
1600번째 기사 완료
1700번째 기사 완료
1800번째 기사 완료
1900번째 기사 완료
2000번째 기사 완료
2100번째 기사 완료
2200번째 기사 완료
2300번째 기사 완료
2400번째 기사 완료
2500번째 기사 완료
2600번째 기사 완료
2700번째 기사 완료
2800번째 기사 완료
2900번째 기사 완료
3000번째 기사 완료
3100번째 기사 완료
3200번째 기사 완료
3300번째 기사 완료
3400번째 기사 완료
3500번째 기사 완료
3600번째 기사 완료
3700번째 기사 완료
3800번째 기사 완료
3900번째 기사 완료
4000번째 기사 완료
4100번째 기사 완료
4200번째 기사 완료
4300번째 기사 완료
4400번째 기사 완료
4500번째 기사 완료
끝! 걸린시간 = 3075.575517654419초


In [34]:
df = pd.DataFrame(list(zip(LINK_OF_ARTICLE, TITLE_OF_ARTICLE, CONTENT_OF_ARTICLES, DATE_OF_ARTICLE)), columns =['LINK','Title', 'Content', 'Date'])
df = df[df.Title != "ERROR"]

In [35]:
df['Content'] = df['Content'].str.replace(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)" , '')
df['Content'] = df['Content'].str.replace('[', '')
df['Content'] = df['Content'].str.replace(']', '')
df['Content'] = df['Content'].str.replace("'", "")  
df['Content'] = df['Content'].str.replace('"', '')
df['Content'] = df['Content'].str.replace(',', '')
df['Content'] = df['Content'].str.replace(r'\\n', '') 
df['Content'] = df['Content'].str.replace(r'\\r', '')
df['Content'] = df['Content'].str.replace(r'\\xa0', '')
df['Content'] = df['Content'].str.replace('‘', '')
df['Content'] = df['Content'].str.replace('’', '')
df['Content'] = df['Content'].str.replace('“', '')
df['Content'] = df['Content'].str.replace('”', '')
df['Content'] = df['Content'].str.replace('//', '')
df['Content'] = df['Content'].str.replace('ㆍ', '')
df['Content'] = df['Content'].str.replace('  ', '')
df['Content'] = df['Content'].str.replace('·', '')
df['Content'] = df['Content'].str.replace('…', '')
df['Content'] = df['Content'].str.replace('〈br〉', '')
df['Content'] = df['Content'].str.replace('「', '')
df['Content'] = df['Content'].str.replace('」', '')
df['Content'] = df['Content'].str.replace(r'\\', '')
df['Content'] = df['Content'].str.lstrip()
df['Content'] = df['Content'].str.replace(r'\(([^)]+)\)','')

df['Content'] = df['Content'].str.replace('[.][.]', '.')

df['Title'] = df['Title'].str.replace(r'\[([^)]+)\]','')
df['Title'] = df['Title'].str.replace('‘', '')
df['Title'] = df['Title'].str.replace('’', '')
df['Title'] = df['Title'].str.replace(',', '')
df['Title'] = df['Title'].str.replace('[', '')
df['Title'] = df['Title'].str.replace(']', '')
df['Title'] = df['Title'].str.replace('…', '')
df['Title'] = df['Title'].str.replace('-', '')
df['Title'] = df['Title'].str.replace('·', '')
df['Title'] = df['Title'].str.replace('“', '')
df['Title'] = df['Title'].str.replace('”', '')
df['Title'] = df['Title'].str.replace('"', '')
df['Title'] = df['Title'].str.replace('···', '')
df['Title'] = df['Title'].str.replace("'", '')
df['Title'] = df['Title'].str.replace('?', '')
df['Title'] = df['Title'].str.replace('\n', '')

#이후 전처리 과정을 한번 더 거칠꺼라 기본적인 부분만 전처리 하였습니다.

In [6]:
df.to_csv(f'경향신문_{keyword}({start_date}~{end_date}).csv', encoding = 'utf-8-sig', index_label = False, index = False)